In [1]:
# Import required modules 
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt 
import os 
import matplotlib.lines as mlines
from datetime import datetime
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FuncFormatter
import gsw # Gibbs-SeaWater (GSW) Oceanographic Toolbox 

### Import data and define variables 

In [2]:
# Figure 4A - Bed & Ice Surface Elevation (BedMachine v5) & Terminus Position
KBS_bed_machine = pd.read_csv('R:/KOGE_BUGT/github/data/BedMachine_profiles/150m/KBS_bed_elevation_150m.csv') 
KBS_distance = KBS_bed_machine['distance']
KBS_bed_elevation = KBS_bed_machine['bed']
KBS_surface_elevation = KBS_bed_machine['surface']
KBS_surface_elevation.replace(0, np.nan, inplace=True)
KBS_terminus_timeseries = pd.read_csv('R:/KOGE_BUGT/github/data/terminus_position_along_flowline/koge_bugt_S_TP_distance_along_flowline.csv')
KBS_terminus_timeseries['Date'] = pd.to_datetime(KBS_terminus_timeseries['Date'], format='%d/%m/%Y')
KBS_terminus_timeseries['DOY'] = KBS_terminus_timeseries['Date'].dt.dayofyear 
KBS_terminus_timeseries['Year'] = KBS_terminus_timeseries['Date'].dt.year

start_year = 2016
end_year = 2024
scale_start = 2
scale_end = 10
years = np.arange(start_year, end_year + 1)
scale_values = np.linspace(scale_start+1.2, scale_end-0.6, len(years))
year_offsets = dict(zip(years, scale_values))
day_scale = 0.7 

KBS_SITES = {'T': (10498.173, 82.95964813232422, '#f77f00'),
             'T3': (7498.595, 335.95458984375, '#a30b37'),
             'T6': (4498.714, 533.7948608398438, '#219ebc'),
             'T9': (1499.990, 783.9451293945312, '#10678e')}

In [3]:
# Figure 4B - Terminus Position 
KBS_terminus = pd.read_csv(r"R:/KOGE_BUGT/github/data/MaQiT_terminus_position_change/koge_bugt_S_maqit_output_2015_2024.csv", skiprows=1, parse_dates= {"Date" : ["Year","Month","Day"]})
KBS_terminus.sort_values(by='Date', ascending = True, inplace = True) 
KBS_january_2016_index = KBS_terminus.index[KBS_terminus['Date'] == '2016-01-12'].tolist()
KBS_terminus['TP_RELATIVE_TO_JAN_2016'] = KBS_terminus['Terminus Position on flowline'] - KBS_terminus['Terminus Position on flowline'].iloc[KBS_january_2016_index[0]]
KBS_jan_2016_label = KBS_terminus['Date'].iloc[11].strftime('%d/%m/%Y')

def y_fmt(x, pos): # Define a function to format the y-axis labels so positive values are prefixed with a '+' sign.
    return f'{x:+.1f}'

In [4]:
# Figure 4C - Ice Discharge (Mankoff et al., 2020)
KBS_discharge_df = pd.read_csv('R:/KOGE_BUGT/github/data/mankoff_2020_ice_discharge/koge_bugt_S_ice_discharge.csv')
KBS_discharge = KBS_discharge_df['Discharge']
KBS_discharge_error = KBS_discharge_df['Error']
KBS_discharge_date = KBS_discharge_df['Date']
KBS_discharge_date = pd.to_datetime(KBS_discharge_date)
date_mask = (KBS_discharge_df['Date'] >= '2016-01-01') & (KBS_discharge_df['Date'] <= '2024-01-01')
mean_discharge = KBS_discharge_df.loc[date_mask, 'Discharge'].mean() # Calculate the mean ice discharge between 2016 and 2024.

In [5]:
# Figure 4D - Ice Velocity (ITS_LIVE)
KBS_velocity_directory = "R:/KOGE_BUGT/github/data/ITS_LIVE_velocity_fahnestock_widget/koge_bugt_south/"
for filename in os.listdir(KBS_velocity_directory):
    if filename.endswith(".csv"):
        file_path = os.path.join(KBS_velocity_directory, filename)
        df = pd.read_csv(file_path)
        df_filtered = df[df[' dt (days)'] <= 12] # Remove any rows where the seperatime time is greater than 12 days. 
        df_filtered.loc[:, 'mid_date'] = pd.to_datetime(df_filtered['mid_date']).dt.strftime('%Y-%m-%d') # Convert the mid_date to %Y-%m-%d' format.
        df_filtered = df_filtered.sort_values(by='mid_date') # Sort the dataframe by date.
        df_filtered = df_filtered[(df_filtered['mid_date'] >= '2016-01-01') & (df_filtered['mid_date'] <= '2024-01-01')]  # Filter date range between 2016 and 2024.
        df_filtered['rolling_avg'] = df_filtered[' v [m/yr]'].rolling(window=30).mean() # Calculate a rolling 30-day average.
        df_filtered.to_csv(file_path, index=False) # Save the filtered dataframe back to the original file path, effectively updating it. 

KBS_velocity_file_paths = {'T': 'R:/KOGE_BUGT/github/data/ITS_LIVE_velocity_fahnestock_widget/koge_bugt_south/lat_64.98708406_lon_-41.18927997.csv',
    'T3': 'R:/KOGE_BUGT/github/data/ITS_LIVE_velocity_fahnestock_widget/koge_bugt_south/lat_65.0006875_lon_-41.24280233.csv',
    'T6': 'R:/KOGE_BUGT/github/data/ITS_LIVE_velocity_fahnestock_widget/koge_bugt_south/lat_65.00123594_lon_-41.30044494.csv',
    'T9': 'R:/KOGE_BUGT/github/data/ITS_LIVE_velocity_fahnestock_widget/koge_bugt_south/lat_64.98626745_lon_-41.34535083.csv'}

sampling_location_colors = {'T': '#f77f00', 'T3': '#a30b37', 'T6': '#219ebc', 'T9': '#10678e'}

In [6]:
# Figure 4E - Surface Runoff (Mankoff, 2020) & Ocean Thermal Forcing 

    # SURFACE RUNOFF
racmo_ds = xr.open_dataset('R:/datasets/Mankoff_2020/RACMO.nc')
mar_ds = xr.open_dataset('R:/datasets/Mankoff_2020/MAR.nc')
start_date = '2016-01-01'
end_date = '2023-12-31'
KBS_basin_ids = [72463, 72457, 72447, 72441, 72430, 72422] 
    ## RACMO ##
KBS_racmo_discharge = racmo_ds['discharge'].sel(station=KBS_basin_ids)
KBS_racmo_discharge_2009_2023 = KBS_racmo_discharge.sel(time=slice(start_date, end_date))
KBS_racmo_cumulative_volume_daily = KBS_racmo_discharge_2009_2023 * 86400 # Convert m3/s to m3/day
KBS_racmo_cumulative_volume_monthly = KBS_racmo_cumulative_volume_daily.resample(time='M').sum()
KBS_racmo_cumulative_volume_monthly_sum = KBS_racmo_cumulative_volume_monthly.sum(dim='station')
    ## MAR ##
KBS_mar_discharge = mar_ds['discharge'].sel(station=KBS_basin_ids)
KBS_mar_discharge_2009_2023 = KBS_mar_discharge.sel(time=slice(start_date, end_date))
KBS_mar_cumulative_volume_daily = KBS_mar_discharge_2009_2023 * 86400 # Convert m3/s to m3/day
KBS_mar_cumulative_volume_monthly = KBS_mar_cumulative_volume_daily.resample(time='M').sum()
KBS_mar_cumulative_volume_monthly_sum = KBS_mar_cumulative_volume_monthly.sum(dim='station')

    # OCEAN THERMAL FORCING
lat_150m = 64.793177
lon_150m = -39.886382
depth_150m = -150
file_path_150m = 'R:/KOGE_BUGT/data/ocean/TOPAZ/150m/cmems_mod_arc_phy_my_topaz4_P1M_1730718237007.csv'

# Read the data for 150m depth
data = pd.read_csv(file_path_150m, skiprows=8)
data['time'] = pd.to_datetime(data['time'])
data = data.dropna(subset=['thetao', 'so'])
thetao_150m = data['thetao'].to_numpy()
practical_salinity_150m = data['so'].to_numpy()

# Calculate pressure, absolute salinity, conservative temperature, in-situ temperature, freezing point, and thermal forcing
pressure_150m = gsw.p_from_z(depth_150m, lat_150m)
absolute_salinity_150m = gsw.SA_from_SP(practical_salinity_150m, pressure_150m, lon_150m, lat_150m)
conservative_temperature_150m = gsw.CT_from_pt(absolute_salinity_150m, thetao_150m)
in_situ_temperature_150m = gsw.t_from_CT(absolute_salinity_150m, conservative_temperature_150m, pressure_150m)
freezing_point_150m = gsw.t_freezing(absolute_salinity_150m, pressure_150m, 0)
thermal_forcing_150m = in_situ_temperature_150m - freezing_point_150m

### Plot Figure 4 - Koge Bugt South

In [ ]:
## Plot Figure 4 ##
print('Plotting Figure 4...')
fig = plt.figure(figsize=(9, 17))
plt.rcParams["font.family"] = "Arial"
gs = GridSpec(6, 1, height_ratios=[1.2, 0.08, 1, 1, 1.2, 1.1])

# Figure 4A - Bed & Ice Surface Elevation (BedMachine v5) & Terminus Position
ax1 = fig.add_subplot(gs[0])
ax1.plot(KBS_distance/1000, KBS_bed_elevation, color='black', label='Bed Elevation')
ax1.plot(KBS_distance/1000, KBS_surface_elevation, color='#023047', label='Surface Elevation')
ax1.set_ylabel('Elevation (m)', labelpad=10, fontsize=14)
ax1.set_xlabel('Distance along flowline (km)', labelpad=6, fontsize=14)
ax1.set_xlim(0, 6)
ax1.set_xticks(range(0, 17, 2))
ax1.set_ylim(-600, 1400)
ax1.set_yticks(range(-500, 1251, 250))
ax1.tick_params(axis='y', labelsize=13)
ax1.tick_params(axis='x', labelsize=13)
ax1.tick_params(axis='x', length=7)
x_values = KBS_distance/1000
y1_values = [-700] * len(KBS_bed_elevation)
y2_values = KBS_bed_elevation
ax1.fill_between(x_values, y1_values, y2_values, color='gray', alpha=0.5, zorder=1)
ax1.fill_between(x_values, KBS_bed_elevation, KBS_surface_elevation, color='#90e0ef', alpha=0.3, zorder=1)
for label, (dist, elev, color) in KBS_SITES.items():
    ax1.scatter(dist/1000, elev, color=color, edgecolor='white', label=label, s=150, zorder=5)
ax1.text(0.98, 0.96, 'a', transform=ax1.transAxes, fontsize=20, fontweight='bold', va='top', ha='right')

ax2 = ax1.twinx()
ax2.set_ylim(scale_start, scale_end)
ax2.set_yticks(scale_values)
ax2.set_yticklabels([str(year) for year in years])
ax2.set_ylabel('Year', rotation=270, labelpad=10, fontsize=14)
ax2.tick_params(axis='y', labelsize=13)
ax2.set_ylabel('Year', labelpad=25, fontsize=14)
for year in years:
    year_data = KBS_terminus_timeseries[KBS_terminus_timeseries['Year'] == year]
    if not year_data.empty:
        tp_distances = year_data['distance_along_line'] / 1000
        tp_offsets = year_offsets[year] + (year_data['DOY'] / 365.0) * day_scale
        ax2.scatter(tp_distances, tp_offsets, c=year_data['DOY'], cmap='viridis', s=10, label=f'Terminus {year}')

cbar_ax = fig.add_axes([0.28, 0.91, 0.45, 0.01]) # [left, bottom, width, height]
cbar = plt.colorbar(ax2.collections[0], cax=cbar_ax, orientation='horizontal', shrink=0.4)
cbar.set_label('Terminus Position (Day of Year)', fontsize=12, labelpad=10)
cbar.ax.tick_params(labelsize=12)
cbar.ax.xaxis.set_label_position('top')

# Figure 4B - Terminus Position
ax3 = fig.add_subplot(gs[2])
ax3.plot(KBS_terminus['Date'], KBS_terminus['TP_RELATIVE_TO_JAN_2016']/1000, color='#009fb7', linewidth=1.25)
ax3.scatter(KBS_terminus['Date'], KBS_terminus['TP_RELATIVE_TO_JAN_2016']/1000, color='#009fb7', s=6, marker='x')
ax3.set_ylabel(f'Terminus position change \n relative to {KBS_jan_2016_label} (km)', labelpad=10, fontsize=14)
ax3.set_ylim(-0.7, 1.2)
ax3.grid(axis='x', linestyle='--', alpha=0.9) 
ax3.set_xlim(pd.to_datetime('2016-01-01'), pd.to_datetime('2024-01-01'))
ax3.axhline(0, color='black', linewidth=0.5, linestyle='-', alpha=0.9) # Add a horizontal line at 0 (representing no change)
ax3.tick_params(axis='y', labelsize=13)
ax3.tick_params(axis='x', labelsize=13, labelbottom=False)
ax3.yaxis.set_major_formatter(FuncFormatter(y_fmt))
ax3.text(0.98, 0.94, 'b', transform=ax3.transAxes, fontsize=20, fontweight='bold', va='top', ha='right')

# Figure 4C - Ice Discharge (Mankoff et al., 2020)
ax4 = fig.add_subplot(gs[3])
ax4.plot(KBS_discharge_date, KBS_discharge, color='#009fb7', linewidth=1.25)
ax4.fill_between(KBS_discharge_date, KBS_discharge - KBS_discharge_error, KBS_discharge + KBS_discharge_error, alpha=0.1, color='#009fb7', zorder=3)
ax4.set_ylabel('Ice Discharge \n (Gt/yr)', labelpad=10, fontsize=14)
ax4.set_xlim(pd.to_datetime('2016-01-01'), pd.to_datetime('2024-01-01'))
ax4.tick_params(axis='y', labelsize=13)
ax4.tick_params(axis='x', labelsize=13, labelbottom=False)
ax4.grid(axis='x', linestyle='--', alpha=0.9)
ax4.axhline(mean_discharge, color='gray', linestyle='-', linewidth=1.2, alpha=0.6)  # Add a horizontal line for the mean ice discharge (2016-2024)
ax4.set_ylim(6.7, 10.3)
ax4.text(0.98, 0.96, 'c', transform=ax4.transAxes, fontsize=20, fontweight='bold', va='top', ha='right')

# Figure 4D - Ice Velocity (ITS_LIVE)
ax5 = fig.add_subplot(gs[4])
for location, path in KBS_velocity_file_paths.items():
    df = pd.read_csv(path)
    df['mid_date'] = pd.to_datetime(df['mid_date']).dt.date  
    ax5.scatter(df['mid_date'], df[' v [m/yr]']/1000, marker='o', linestyle='None', s=4, label=f'{location}', color=sampling_location_colors[location], alpha=0.8)
ax5.set_ylabel('Ice velocity (km/yr)', labelpad=15, fontsize=14)
ax5.grid(axis='x', linestyle='--', alpha=0.9)
ax5.set_xlim(pd.to_datetime('2016-01-01'), pd.to_datetime('2024-01-01'))
ax5.tick_params(axis='y', labelsize=13)
ax5.tick_params(axis='x', labelsize=13, labelbottom=False)
ax5.set_ylim(-0.15, 5.5)
ax5.legend(loc='upper left', bbox_to_anchor=(0.025, 0.98), ncol=4, frameon=True, framealpha=1, fontsize=13, markerscale=2.8, columnspacing=0.7, handletextpad=0.5, fancybox=True, borderaxespad=0.8, edgecolor='gray', handlelength=1.1)
ax5.text(0.98, 0.94, 'd', transform=ax5.transAxes, fontsize=20, fontweight='bold', va='top', ha='right')

# Figure 4E - Surface Runoff and Ocean Thermal Forcing
ax6 = fig.add_subplot(gs[5]) 
ax6.plot(KBS_mar_cumulative_volume_monthly_sum.time, KBS_mar_cumulative_volume_monthly_sum, label='MAR', linewidth=1.5, color='#023047')
ax6.plot(KBS_racmo_cumulative_volume_monthly_sum.time, KBS_racmo_cumulative_volume_monthly_sum, label='RACMO', linewidth=1.5, color='#4ba3c3')
ax6.set_ylabel('Cumulative Monthly \n Surface Runoff (m³)', labelpad=10, fontsize=14)
ax6.grid(axis='x', linestyle='--', alpha=0.9)
ax6.set_ylim(0, 1075000000)
ax6.set_xlim(pd.to_datetime('2016-01-01'), pd.to_datetime('2024-01-01'))
ax6.text(0.98, 0.96, 'e', transform=ax6.transAxes, fontsize=20, fontweight='bold', va='top', ha='right')
ax6.tick_params(axis='y', labelsize=13)
ax6.tick_params(axis='x', labelsize=13)
ax6.set_xlabel('Year', labelpad=10, fontsize=14)

ax6_twin = ax6.twinx()
ax6_twin.plot(data['time'], thermal_forcing_150m, label='TOPAZ4B', color='#bd1f36', linewidth=1.5)
ax6_twin.scatter(data['time'], thermal_forcing_150m, color='#bd1f36', s=15)
ax6_twin.set_ylabel('Ocean Thermal \n Forcing (°C)', rotation=270, labelpad=40, fontsize=14)
ax6_twin.tick_params(axis='y', labelsize=13)
ax6_twin.set_ylim(0.5, 7.5)
ax6_twin.set_yticks(range(1, 8)) 

handles1, labels1 = ax6.get_legend_handles_labels()  
handles2, labels2 = ax6_twin.get_legend_handles_labels()  
ax6.legend(handles1 + handles2, labels1 + labels2, loc='upper left', bbox_to_anchor=(0.22, 1.02), ncol=3, frameon=True, framealpha=1, fontsize=13, markerscale=2.2, columnspacing=0.7, handletextpad=0.5, fancybox=True, borderaxespad=0.8, edgecolor='gray', handlelength=1.1)

## SAVE AND PRINT FIGURE 4 ##
fig.add_subplot(gs[1]).axis('off')
plt.subplots_adjust(hspace=0.15)
plt.savefig('R:/KOGE_BUGT/github/figures/Figure_4_Koge_Bugt_South.png', dpi=300, bbox_inches='tight')
plt.show()

Plotting Figure 4...
